# Preprocessing the Dataset

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

Importing all required packages

In [ ]:
import pandas as pd
import sys
import pickle

sys.path.append('./')
import utils

from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
seed = 42

## Loading Rogue Wave Data

Note:

- The data are stored in a csv file of the size 1048574 x 29.
- Modified ina's input dataset, now the encoded values as well as original wave dir and obj weather data are in a same file

In [ ]:
data_rogue_waves = pd.read_csv("ML_Matrix_full_encoded_st.csv", index_col=0)

In [ ]:
print(f"The dataset has {data_rogue_waves.shape[1] - 1} variables that describe {data_rogue_waves.shape[0]} waves.")

In [ ]:
data_rogue_waves.head()

### Preprocessing

Some of the target values are close to 3, which is not realistic, hence, they are excluded from the dataset.

In [ ]:
thr_target = 2.7
data_rogue_waves = data_rogue_waves.loc[data_rogue_waves.AI_10min < thr_target]

Select key features

In [ ]:
selected_features = ['H_s', 'lambda_40', 'lambda_30', 'L_deep', 's', 'mu', 'kh', 'T_p', 'eps', 'nu', 'Q_p', 'swell', 'v_wind', 'T_air', 'Delta_T', 'p', 'Delta_p_1h','AI_10min']
data_rogue_waves = data_rogue_waves.loc[:,selected_features]

In [ ]:
data_rogue_waves.head()

In [ ]:
utils.plot_distributions(dataset=data_rogue_waves, ncols=5)

For classification approach, we need to turn the target values into classes.  
We will have a look at three different target transformations:

- case 1: class 0: target < 2.0 and class 1: target > 2.0 
- case 2: class 0: target < 1.5 and class 1: target > 2.0
- case 3: class 0: target < 1.5, class 1: 1.5 < target < 2.0 and class 2: target > 2.0

**Case 1:**

In [ ]:
# Define the threshold
threshold = 2.0

# Binarize target 
data_rogue_waves_case1 = data_rogue_waves.copy()
data_rogue_waves_case1['target'] = data_rogue_waves_case1['AI_10min'].apply(lambda x: 0 if x < threshold else 1)
data_rogue_waves_case1['target'] = data_rogue_waves_case1['target'].astype(int)

print('Dataset target distribution:')
print(Counter(data_rogue_waves_case1['target']))

**Case 2:**

In [ ]:
# Define thresholds for three classes
threshold_1 = 1.5
threshold_2 = 2.0

# Binarize target 
data_rogue_waves_case2 = data_rogue_waves.copy()
data_rogue_waves_case2['target'] = data_rogue_waves_case2['AI_10min'].apply(lambda x: 0 if x < threshold_1 else (0.5 if x < threshold_2 else 1))
data_rogue_waves_case2 = data_rogue_waves_case2[data_rogue_waves_case2['target'].isin([0, 1])]
data_rogue_waves_case2['target'] = data_rogue_waves_case2['target'].astype(int)

print('Dataset target distribution:')
print(Counter(data_rogue_waves_case2['target']))

**Case 3:**

In [ ]:
# Define thresholds for three classes
threshold_1 = 1.5
threshold_2 = 2.0

# Binarize target 
data_rogue_waves_case3 = data_rogue_waves.copy()
data_rogue_waves_case3['target'] = data_rogue_waves_case3['AI_10min'].apply(lambda x: 0 if x < threshold_1 else (1 if x < threshold_2 else 2))
data_rogue_waves_case3['target'] = data_rogue_waves_case3['target'].astype(int)

print('Dataset target distribution:')
print(Counter(data_rogue_waves_case3['target']))

In [ ]:
col_target = "target"
col_features = data_rogue_waves_case1.columns.drop(col_target).to_list()

### Setup train and test data

Split data into train and test dataset with stratified split, to ensure having enough instances of the minority class in both train and test set. Then save the dataset to pickl.

In [ ]:
for i, data in enumerate([data_rogue_waves_case1, data_rogue_waves_case2, data_rogue_waves_case3]):

    # We will first separate the target and feature columns.
    X = data[col_features]
    y = data[col_target]

    # split into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, train_size=0.80, random_state=seed)

    X_train.reset_index(inplace=True, drop=True)
    X_test.reset_index(inplace=True, drop=True)
    y_train.reset_index(inplace=True, drop=True)
    y_test.reset_index(inplace=True, drop=True)

    print('Training dataset target distribution:')
    print(Counter(y_train))

    print('Test dataset target distribution:')
    print(Counter(y_test))

    # Save the model with joblib
    data_and_model = [X_train, X_test, y_train, y_test]

    with open(f'./data_case{i+1}.pickle', 'wb') as handle:
        pickle.dump(data_and_model, handle, protocol=pickle.HIGHEST_PROTOCOL)